In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM Layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Fully Connected Layer (Output Layer)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: (batch_size, seq_length, hidden_size)

        # Take the last time step's output
        out = self.fc(out[:, -1, :])  # Shape: (batch_size, output_size)
        
        return out


In [3]:
# Generate synthetic sequential data for regression
np.random.seed(42)
torch.manual_seed(42)

# Create dummy time-series data
def generate_data(seq_length=10, num_samples=100):
    X = np.array([np.arange(i, i+seq_length) for i in range(num_samples)])
    y = np.sum(X, axis=1)  # Example: sum of the sequence as target
    return torch.tensor(X, dtype=torch.float32).unsqueeze(-1), torch.tensor(y, dtype=torch.float32).unsqueeze(-1)

# Prepare dataset
seq_length = 10
X, y = generate_data(seq_length=seq_length, num_samples=1000)

# Split into train and test sets
train_size = int(0.8 * len(X))
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]


In [17]:
# Model Hyperparameters
input_size = 1
hidden_size = 10
num_layers = 2
output_size = 1
learning_rate = 0.1
num_epochs = 500

# Initialize Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)

# Loss and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Move data to device
X_train, y_train, X_test, y_test = X_train.to(device), y_train.to(device), X_test.to(device), y_test.to(device)

# Training Loop
for epoch in range(num_epochs):
    model.train()
    
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss every 10 epochs
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/500], Loss: 21598604.0000
Epoch [20/500], Loss: 21532242.0000
Epoch [30/500], Loss: 21465208.0000
Epoch [40/500], Loss: 21398420.0000
Epoch [50/500], Loss: 21332180.0000
Epoch [60/500], Loss: 21266502.0000
Epoch [70/500], Loss: 21201308.0000
Epoch [80/500], Loss: 21136546.0000
Epoch [90/500], Loss: 21072158.0000
Epoch [100/500], Loss: 21008104.0000
Epoch [110/500], Loss: 20944360.0000
Epoch [120/500], Loss: 20880902.0000
Epoch [130/500], Loss: 20817716.0000
Epoch [140/500], Loss: 20754780.0000
Epoch [150/500], Loss: 20692092.0000
Epoch [160/500], Loss: 20629642.0000
Epoch [170/500], Loss: 20567422.0000
Epoch [180/500], Loss: 20505426.0000
Epoch [190/500], Loss: 20443648.0000
Epoch [200/500], Loss: 20382088.0000
Epoch [210/500], Loss: 20320738.0000
Epoch [220/500], Loss: 20259592.0000
Epoch [230/500], Loss: 20198656.0000
Epoch [240/500], Loss: 20137922.0000
Epoch [250/500], Loss: 20077388.0000
Epoch [260/500], Loss: 20017050.0000
Epoch [270/500], Loss: 19956910.0000
Epoch [280

In [18]:
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    test_loss = criterion(predictions, y_test)

print(f'Test Loss: {test_loss.item():.4f}')


Test Loss: 75069544.0000


Just for understanding how LSTM works!!